# Heated Wall in Simple configuration - Verification sheet

Verification Test for the XNSFE_Solver

Setup:
  * Interface at 90°.  
  * Equal fluid densities => simplified setting  
  * Also no Heat capacity => infinitely fast heat conduction  
  * Reference Simulation done with FastMarching

## Load BoSSS

In [ ]:
#r "../../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Setup Workflowmanagement, Batchprocessor and Database

In [ ]:
ExecutionQueues

In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();

In [ ]:
static var myDb = myBatch.CreateOrOpenCompatibleDatabase("XNSFE_HeatedWall");

In [ ]:
myDb.Path

In [ ]:
var date = System.DateTime.Today;
string ProjectName = $"HeatedWallSimple_VerificationFastMarching_{date.Year}{date.Month}{date.Day}";

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName);

## Setup Simulationcontrols

In [ ]:
using BoSSS.Application.XNSFE_Solver;

In [ ]:
int[] hRes = {4};//{1, 2, 3, 4};
int[] pDeg = {2};//{ 1, 2, 3, 4};
double[] Q = {0.0, 0.05, 0.1, 0.2};

In [ ]:
List<XNSFE_Control> Controls = new List<XNSFE_Control>();
foreach(int h in hRes){
    foreach(int p in pDeg){
        foreach(double q in Q){

        var ctrl = new XNSFE_Control();
        ctrl.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("HeatFlux", q));

        ctrl.DbPath      = null;
        ctrl.SessionName = $"HeatedWall_Simple_res:{h}_p:{p}_Q:{q}";
        ctrl.ProjectName = $"ProjectName";
        ctrl.SetDatabase(myDb);
        ctrl.savetodb = true;        

        ctrl.SetDGdegree(p);

        // For Gravity the above routine does not contain standard values, add the options manually
        ctrl.FieldOptions.Add("GravityX#A", new FieldOpts() {
            SaveToDB = FieldOpts.SaveToDBOpt.TRUE
        });
        ctrl.FieldOptions.Add("GravityY#A", new FieldOpts() {
            SaveToDB = FieldOpts.SaveToDBOpt.TRUE
        });
        ctrl.FieldOptions.Add("GravityX#B", new FieldOpts() {
            SaveToDB = FieldOpts.SaveToDBOpt.TRUE
        });
        ctrl.FieldOptions.Add("GravityY#B", new FieldOpts() {
            SaveToDB = FieldOpts.SaveToDBOpt.TRUE
        });        

        #region grid
        double L = 5.0;
        int kelemR = h;
        string[] Bndy = new string[] {  "Inner",
                                        "NavierSlip_linear_ConstantHeatFlux_right",
                                        "pressure_outlet_ZeroGradient_top",
                                        "freeslip_ZeroGradient_left",
                                        "pressure_outlet_ZeroGradient_bottom"};

        ctrl.GridFunc = delegate () {
            double[] Xnodes = GenericBlas.Linspace(-L, 0, kelemR + 1);
            double[] Ynodes = GenericBlas.Linspace(0, 3 * L, 3 * kelemR + 1);
            var grd = Grid2D.Cartesian2DGrid(Xnodes, Ynodes);

            for(byte i= 1; i < Bndy.Count(); i++) {
                grd.EdgeTagNames.Add(i, Bndy[i]);
            }

            grd.DefineEdgeTags(delegate (double[] X) {
                byte et = 0;
                if(Math.Abs(X[0] - Xnodes.Last()) < 1e-8)
                    return 1;
                if(Math.Abs(X[0] - Xnodes.First()) < 1e-8)
                    return 3;
                if(Math.Abs(X[1] - Ynodes.Last()) < 1e-8)
                    return 2;
                if(Math.Abs(X[1] - Ynodes.First()) < 1e-8)
                    return 4;
                return et;
            });

            return grd;
        };
        #endregion

        #region material
        ctrl.PhysicalParameters = new BoSSS.Solution.XNSECommon.PhysicalParameters() {
            rho_A = 1.0, // 958.0
            rho_B = 1.0, // 0.59,

            mu_A = 1, //2.82 * 1e-4,
            mu_B = 0.001, //1.23 * 1e-6,

            Sigma = 1.0,
            betaS_A = 1000, // sliplength is mu/beta
            betaS_B = 1000,
        };

        ctrl.ThermalParameters = new BoSSS.Solution.XheatCommon.ThermalParameters() {
            rho_A = 1.0, // 958.0
            rho_B = 1.0, //0.59,

            k_A = 1.0, // 0.6
            k_B = 1.0, // 0.026,

            c_A = 0.0,
            c_B = 0.0,

            hVap = 1,//2.257 * 1e6,
            T_sat = 0.0 // 373.0
        };

        ctrl.PhysicalParameters.IncludeConvection = true;
        ctrl.ThermalParameters.IncludeConvection = true;
        ctrl.PhysicalParameters.Material = false;
        #endregion

        #region Initial Condition - Exact Solution

        // solution for massflux and velocity at level set
        double y0 = 0.2 * L;

        // inital values
        double g = 4;
        ctrl.AddInitialValue("Phi", $"(X, t) => -{y0} + X[1]", true);
        ctrl.AddInitialValue("Temperature#A", $"(X, t) => {ctrl.ThermalParameters.T_sat}", true);
        ctrl.AddInitialValue("Temperature#B", $"(X, t) => {ctrl.ThermalParameters.T_sat}", true);
        ctrl.AddInitialValue("GravityY#A", $"(X, t) => -{g}", true);

        #endregion

        #region Boundary Conditions

        double v = 1.0;
        ctrl.AddBoundaryValue(Bndy[1], "HeatFluxX#A", $"(X, t) => {q}", true);
        ctrl.AddBoundaryValue(Bndy[1], "VelocityY#A", $"(X, t) => {v}", true);
        ctrl.AddBoundaryValue(Bndy[1], "VelocityY#B", $"(X, t) => {v}", true);


        ctrl.AddBoundaryValue(Bndy[3]);
        ctrl.AddBoundaryValue(Bndy[2]);
        ctrl.AddBoundaryValue(Bndy[4], "Pressure#A", $"(X, t) => {y0} * {ctrl.PhysicalParameters.rho_A} * {g}", true);

        #endregion

        #region AMR

        int level = 3;
        ctrl.AdaptiveMeshRefinement = level > 0;
        ctrl.activeAMRlevelIndicators.Add(new BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater.AMRonNarrowband() { maxRefinementLevel = level });
        ctrl.AMR_startUpSweeps = level;

        #endregion

        #region Timestepping

        ctrl.AdvancedDiscretizationOptions.SST_isotropicMode = BoSSS.Solution.XNSECommon.SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
        ctrl.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.FastMarching;
        ctrl.Timestepper_LevelSetHandling =  BoSSS.Solution.XdgTimestepping.LevelSetHandling.LieSplitting;

        ctrl.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
        ctrl.NonLinearSolver.Globalization = BoSSS.Solution.AdvancedSolvers.Newton.GlobalizationOption.Dogleg;
        ctrl.NonLinearSolver.ConvergenceCriterion = 1e-8;
        ctrl.NonLinearSolver.MaxSolverIterations = 10;

        ctrl.SkipSolveAndEvaluateResidual = false;

        ctrl.TimeSteppingScheme = BoSSS.Solution.XdgTimestepping.TimeSteppingScheme.ImplicitEuler;
        ctrl.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Transient;
        ctrl.dtFixed = 0.01;
        ctrl.Endtime = 15.0;
        ctrl.NoOfTimesteps = (int)(ctrl.Endtime / ctrl.dtFixed);

        #endregion
        ctrl.PostprocessingModules.Add(new BoSSS.Application.XNSFE_Solver.PhysicalBasedTestcases.MassfluxLogging() { LogPeriod = 1 });
        ctrl.PostprocessingModules.Add(new BoSSS.Application.XNSFE_Solver.PhysicalBasedTestcases.MovingContactLineLogging() { LogPeriod = 1 });

        Controls.Add(ctrl);
        }
    }
}

In [ ]:
Controls.Count

## Start simulations on Batch processor

In [ ]:
foreach(var C in Controls) {
    Type solver = typeof(BoSSS.Application.XNSFE_Solver.XNSFE<XNSFE_Control>);

    string jobName       = C.SessionName;
    var oneJob           = new Job(jobName, solver);
    oneJob.NumberOfMPIProcs = 1;    
    oneJob.SetControlObject(C);
    oneJob.Activate(myBatch, true);
}

### Wait until jobs complete

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(86400 , 60); // wait for one day, or until all jobs finish

## Postprocessing

Track the amount of errors occuring

In [ ]:
int Errors = 0;

### Load sessions

In [ ]:
var sessions = BoSSSshell.WorkflowMgm.Sessions.Where(s => s.SuccessfulTermination == true).ToList();

In [ ]:
sessions

In [ ]:
if(sessions.Count < 4){
    Console.WriteLine("\x1b[31mError: expected 4 successful sessions!\x1b[0m");
    Errors += 4 - sessions.Count;
}

#### Load data

In [ ]:
List<Plot2Ddata> dataCL, dataEvap;

try{
    dataCL = sessions.ReadLogDataForMovingContactLine()[1];
    dataEvap = sessions.ReadLogDataForXNSE("Massflux");
} catch {
    Console.WriteLine("\x1b[31mError: loading data!\x1b[0m");
    Errors++;
}

### Process data

In [ ]:
try{
    dataCL.Pick(4).dataGroups.ForEach(s => s.Values = s.Values.Select(v => 180 - v ).ToArray());

    Plot2Ddata evapFluxes = new Plot2Ddata();
    evapFluxes.dataGroups = new Plot2Ddata.XYvalues[dataEvap.Pick(3).dataGroups.Length]; // same as total evaporative heatflux * dt, as rho = hvap = 1
    Plot2Ddata bndFluxes = new Plot2Ddata();
    bndFluxes.dataGroups = new Plot2Ddata.XYvalues[dataCL.Pick(1).dataGroups.Length]; 
    Plot2Ddata.XYvalues[] Vals = new Plot2Ddata.XYvalues[dataCL.Pick(4).dataGroups.Length];
    Plot2Ddata clVeloY = new Plot2Ddata();
    clVeloY.dataGroups = new Plot2Ddata.XYvalues[dataCL.Pick(3).dataGroups.Length]; 
    int dashType = 0;
    for(int i = 0; i<sessions.Count;i++){
        sessions[i].KeysAndQueries.TryGetValue("id:PressureRatio", out object color);
        color = color == null ? 7 : Convert.ToInt32(color);
        dataCL.ForEach(p => p.dataGroups[i].Name = "q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]));
        dataEvap.ForEach(p => p.dataGroups[i].Name = "q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]));
        PlotFormat PF = new PlotFormat(lineColor: (LineColors)color, dashType: ((DashTypes)(++dashType)));
        PF.LineWidth = 3.0;
        dataCL.ForEach(p => p.dataGroups[i].Format = PF.CloneAs());
        dataEvap.ForEach(p => p.dataGroups[i].Format = PF.CloneAs());
        
        double alpha = 180/Math.PI * 0.5*Math.Asin(2*Convert.ToDouble(sessions[i].KeysAndQueries["id:HeatFlux"]));
        Vals[i] = new Plot2Ddata.XYvalues("q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]), new double[] {0.0, 15}, new double[] {alpha, alpha});
        Vals[i].Format = new PlotFormat("b-");

        double dt = Convert.ToDouble(sessions[i].KeysAndQueries["dtFixed"]);
        PF.LineWidth = 3.0;
        PF.LineColor = (LineColors)7;
        PF.DashType = (DashTypes)1;
        evapFluxes.dataGroups[i] = new Plot2Ddata.XYvalues("");
        evapFluxes.dataGroups[i].Values = dataEvap.Pick(3).dataGroups[i].Values.Select(x => x).ToArray();
        evapFluxes.dataGroups[i].Abscissas = dataEvap.Pick(3).dataGroups[i].Abscissas.Select(x => x).ToArray();
        evapFluxes.dataGroups[i].Name = "Q-Evap:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
        evapFluxes.dataGroups[i].Format = PF.CloneAs();
        evapFluxes.dataGroups[i].Values = evapFluxes.dataGroups[i].Values.Select(v => v / dt).ToArray();

        PF.LineWidth = 3.0;
        PF.LineColor = (LineColors)2;
        PF.DashType = (DashTypes)2;
        bndFluxes.dataGroups[i] = new Plot2Ddata.XYvalues("");
        bndFluxes.dataGroups[i].Values = dataCL.Pick(1).dataGroups[i].Values.Select(x => x).ToArray();
        bndFluxes.dataGroups[i].Abscissas = dataCL.Pick(1).dataGroups[i].Abscissas.Select(x => x).ToArray();
        bndFluxes.dataGroups[i].Name = "Q-Bnd:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
        bndFluxes.dataGroups[i].Format = PF.CloneAs();
        bndFluxes.dataGroups[i].Values = bndFluxes.dataGroups[i].Values.Select(v => v * Convert.ToDouble(sessions[i].KeysAndQueries["id:HeatFlux"])).ToArray();

        PF.LineColor = (LineColors)color;
        PF.DashType = (DashTypes)dashType;
        clVeloY.dataGroups[i] = new Plot2Ddata.XYvalues("");
        clVeloY.dataGroups[i].Values = new double[dataCL.Pick(3).dataGroups[i].Values.Length];
        for(int j = 0; j<dataCL.Pick(3).dataGroups[i].Values.Length; j++){
            clVeloY.dataGroups[i].Values[j] = dataCL.Pick(3).dataGroups[i].Values[j] - dataCL.Pick(2).dataGroups[i].Values[j] / Math.Tan(Math.PI / 180.0 * dataCL.Pick(4).dataGroups[i].Values[j]);
        } 
        clVeloY.dataGroups[i].Abscissas = dataCL.Pick(1).dataGroups[i].Abscissas.Select(x => x).ToArray();
        clVeloY.dataGroups[i].Name = "q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
        clVeloY.dataGroups[i].Format = PF.CloneAs();
    }
    dataCL.Pick(4).dataGroups = dataCL.Pick(4).dataGroups.ToList().Cat(Vals).ToArray();    
    evapFluxes = evapFluxes.Merge(bndFluxes);
    dataEvap[3] = evapFluxes; // reassign...
    dataCL[3] = clVeloY;
} catch {
    Console.WriteLine("\x1b[31mError: processing data!\x1b[0m");
    Errors++;
}

### Comparison to reference data

#### (De)Serialization of reference values

In [ ]:
using System.IO;
using Newtonsoft.Json;

In [ ]:
JsonSerializer formatter = new JsonSerializer() {
    NullValueHandling = NullValueHandling.Ignore,
    TypeNameHandling = TypeNameHandling.Auto,
    ConstructorHandling = ConstructorHandling.AllowNonPublicDefaultConstructor,
    ReferenceLoopHandling = ReferenceLoopHandling.Error,
    Formatting = Formatting.Indented
};            

//originally called to write out reference data
// using(var tw = new StringWriter()) {
//     using(JsonWriter writer = new JsonTextWriter(tw)) {
//         formatter.Serialize(writer, dataCL);
//         string Ret = tw.ToString(); 
//         File.WriteAllText(".\\ReferenceData\\dataCL_Reference.json", Ret); 
//     }
// }  
// using(var tw = new StringWriter()) {
//     using(JsonWriter writer = new JsonTextWriter(tw)) {          
//         formatter.Serialize(writer, dataEvap);
//         string Ret = tw.ToString(); 
//         File.WriteAllText(".\\ReferenceData\\dataEvap_Reference.json", Ret);   
//     }    
// }

In [ ]:
List<Plot2Ddata> dataCL_Ref, dataEvap_Ref;

using(var tr = new StringReader(File.ReadAllText(".\\ReferenceData\\dataCL_Reference.json"))) {        
    using(JsonReader reader = new JsonTextReader(tr)) {
            var obj = formatter.Deserialize(reader, typeof(List<Plot2Ddata>));
            dataCL_Ref = (List<Plot2Ddata>)obj;
        }         
}
using(var tr = new StringReader(File.ReadAllText(".\\ReferenceData\\dataEvap_Reference.json"))) {        
    using(JsonReader reader = new JsonTextReader(tr)) {
            var obj = formatter.Deserialize(reader, typeof(List<Plot2Ddata>));
            dataEvap_Ref = (List<Plot2Ddata>)obj;
        }         
}

#### Setup Plot comparison

Left - current data  
Right - reference data

In [ ]:
var gp = new Gnuplot();
gp.SetMultiplot(4, 2);
Plot2Ddata data;

In [ ]:
try{
    gp.SetSubPlot(0, 0);

    data = dataCL.Pick(4);
    data.Xlabel = "time [s]";
    data.Ylabel = "angle [°]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact Angle";

    data.ToGnuplot(gp);

    gp.SetSubPlot(0, 1);

    data = dataCL_Ref.Pick(4);
    for(int i=0; i<data.dataGroups.Count(); i++) { data.dataGroups[i].Format = dataCL.Pick(4).dataGroups[i].Format; }
    data.Xlabel = "time [s]";
    data.Ylabel = "angle [°]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact Angle Reference";

    data.ToGnuplot(gp);
} catch {
    Console.WriteLine("\x1b[31mError: plotting data!\x1b[0m");
    Errors++;
}

In [ ]:
try{
    gp.SetSubPlot(1, 0);

    data = dataEvap.Pick(3);
    data.Xlabel = "time [s]";
    data.Ylabel = "Total Heat Flux [W]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Heatflux";
    data.ShowLegend = false;

    data.ToGnuplot(gp);

    gp.SetSubPlot(1, 1);

    data = dataEvap_Ref.Pick(3);
    for(int i=0; i<data.dataGroups.Count(); i++) { data.dataGroups[i].Format = dataEvap.Pick(3).dataGroups[i].Format; }
    data.Xlabel = "time [s]";
    data.Ylabel = "Total Heat Flux [W]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Heatflux Reference";
    data.ShowLegend = false;

    data.ToGnuplot(gp);
} catch {
    Console.WriteLine("\x1b[31mError: plotting data!\x1b[0m");
    Errors++;
}

In [ ]:
try{
    gp.SetSubPlot(2, 0);

    data = dataCL.Pick(1);
    data.Xlabel = "time [s]";
    data.Ylabel = "Pos Y [m]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Position";
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);

    gp.SetSubPlot(2, 1);

    data = dataCL_Ref.Pick(1);
    for(int i=0; i<data.dataGroups.Count(); i++) { data.dataGroups[i].Format = dataCL.Pick(1).dataGroups[i].Format; }
    data.Xlabel = "time [s]";
    data.Ylabel = "Pos Y [m]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Position Reference";
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);
} catch {
    Console.WriteLine("\x1b[31mError: plotting data!\x1b[0m");
    Errors++;
}

In [ ]:
try{
    gp.SetSubPlot(3, 0);

    data = new Plot2Ddata();
    // Filter the velocity for a smoother representation
    foreach(var group in dataCL.Pick(3).dataGroups){
        Plot2Ddata.XYvalues val = new Plot2Ddata.XYvalues(group.Name, group.Abscissas, group.Values);
        double n, m;
        for(int i = 1; i< val.Values.Count()-1; i++){
            m = val.Values[i + 1];
            n = val.Values[i - 1];
            double mid = 0.5 * (n+m);        
            if(Math.Abs(val.Values[i] - n) > Math.Abs(m - n) )
                val.Values[i] = mid;
        }
        val.Format = group.Format;
        data.dataGroups = data.dataGroups.Cat(val);
    }
    data.Xlabel = "time [s]";
    data.Ylabel = "V_y [m/s]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Velocity";
    data.LegendAlignment = new string[] {"i","l","b"};

    data.ToGnuplot(gp);

    gp.SetSubPlot(3, 1);

    data = new Plot2Ddata();
    // Filter the velocity for a smoother representation
    foreach(var group in dataCL_Ref.Pick(3).dataGroups){
        Plot2Ddata.XYvalues val = new Plot2Ddata.XYvalues(group.Name, group.Abscissas, group.Values);
        double n, m;
        for(int i = 1; i< val.Values.Count()-1; i++){
            m = val.Values[i + 1];
            n = val.Values[i - 1];
            double mid = 0.5 * (n+m);        
            if(Math.Abs(val.Values[i] - n) > Math.Abs(m - n) )
                val.Values[i] = mid;
        }
        val.Format = group.Format;
        data.dataGroups = data.dataGroups.Cat(val);
    }
    for(int i=0; i<data.dataGroups.Count(); i++) { data.dataGroups[i].Format = dataCL.Pick(3).dataGroups[i].Format; }
    data.Xlabel = "time [s]";
    data.Ylabel = "V_y [m/s]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Velocity Reference";
    data.LegendAlignment = new string[] {"i","l","b"};

    data.ToGnuplot(gp);
} catch {
    Console.WriteLine("\x1b[31mError: plotting data!\x1b[0m");
    Errors++;
}

#### Plot comparison

In [ ]:
gp.PlotSVG(1600, 4 * 600)

### Quantitative comparison

#### Assert that the deviance is within limits

First, create some error plots, as the simulations should use the same fixed timestep this is possible.

In [ ]:
List<Plot2Ddata> dataCL_Err = new List<Plot2Ddata>();
List<Plot2Ddata> dataEvap_Err = new List<Plot2Ddata>();

for(int i = 0; i < dataCL.Count(); i++){
    Plot2Ddata errData = new Plot2Ddata();
    for(int j = 0; j < dataCL[i].dataGroups.Count() & j < 4; j++){
        var group = dataCL[i].dataGroups[j];
        var group_ref = dataCL_Ref[i].dataGroups[j];
        Plot2Ddata.XYvalues val = new Plot2Ddata.XYvalues(group.Name, group.Abscissas, group.Values);
        for(int k = 0; k< val.Values.Count(); k++){
            val.Values[k] = group_ref.Values[k] - group.Values[k];
        }
        val.Format = group.Format;
        errData.dataGroups = errData.dataGroups.Cat(val);
    }
    dataCL_Err.Add(errData);
}

for(int i = 0; i < dataEvap.Count(); i++){
    Plot2Ddata errData = new Plot2Ddata();
    for(int j = 0; j < dataEvap[i].dataGroups.Count() & j < 4; j++){
        var group = dataEvap[i].dataGroups[j];
        var group_ref = dataEvap_Ref[i].dataGroups[j];
        Plot2Ddata.XYvalues val = new Plot2Ddata.XYvalues(group.Name, group.Abscissas, group.Values);
        for(int k = 0; k< val.Values.Count(); k++){
            val.Values[k] = group_ref.Values[k] - group.Values[k];
        }
        val.Format = group.Format;
        errData.dataGroups = errData.dataGroups.Cat(val);
    }
    dataEvap_Err.Add(errData);
}

if(dataEvap[0].dataGroups[0].Abscissas.Count() != dataEvap_Ref[0].dataGroups[0].Abscissas.Count() | dataCL[0].dataGroups[0].Abscissas.Count() != dataCL_Ref[0].dataGroups[0].Abscissas.Count()){
    Console.WriteLine("\x1b[31mError: datasets of different lengths!\x1b[0m");
    Errors++;
}

In [ ]:
var gp = new Gnuplot();
gp.SetMultiplot(2,2);

In [ ]:
try{
    gp.SetSubPlot(0, 0);

    data = dataCL_Err.Pick(4);
    data.Xlabel = "time [s]";
    data.Ylabel = "angle [°]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact Angle";
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);

    gp.SetSubPlot(0, 1);

    data = dataEvap_Err.Pick(3);
    data.Xlabel = "time [s]";
    data.Ylabel = "Total Heat Flux [W]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Heatflux";
    data.ShowLegend = true;
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);

    gp.SetSubPlot(1, 0);

    data = dataCL_Err.Pick(1);
    data.Xlabel = "time [s]";
    data.Ylabel = "Pos Y [m]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Position";
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);

    gp.SetSubPlot(1, 1);

    data = dataCL_Err.Pick(3);
    data.Xlabel = "time [s]";
    data.Ylabel = "V_y [m/s]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Velocity Reference";
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);
} catch {
    Console.WriteLine("\x1b[31mError: plotting data!\x1b[0m");
    Errors++;
}

In [ ]:
gp.PlotSVG(1600, 1200)

#### Calculate relative Error Norm
$ \varepsilon_{abs} = ||u - u_{ref}|| = (\sum (u_i - u_{i,ref})^2)^{\frac{1}{2}} $  

$ ||u_{ref}|| = (\sum (u_{i,ref})^2)^{\frac{1}{2}} $  

$ \varepsilon_{rel} = \frac{||u - u_{ref}||}{||u_{ref}||} $  

In [ ]:
int N_end = 1200; // endtime 12 a dt 0.01
Dictionary<string, List<double>> errCL_abs      = new Dictionary<string, List<double>>();
Dictionary<string, List<double>> errEvap_abs    = new Dictionary<string, List<double>>();
Dictionary<string, List<double>> normCL_ref     = new Dictionary<string, List<double>>();
Dictionary<string, List<double>> normEvap_ref   = new Dictionary<string, List<double>>();
Dictionary<string, List<double>> errCL_rel      = new Dictionary<string, List<double>>();
Dictionary<string, List<double>> errEvap_rel    = new Dictionary<string, List<double>>();

In [ ]:
for(int i = 0; i < dataCL_Ref.Count(); i++){
    for(int j = 0; j < dataCL_Ref[i].dataGroups.Count() & j < 4; j++){
        string name = dataCL_Ref[i].dataGroups[j].Name;
        var vals_ref = dataCL_Ref[i].dataGroups[j].Values;
        var vals = dataCL[i].dataGroups[j].Values;
        double norm = 0.0, err = 0.0;
        for(int k = 0; k< N_end; k++){
            err += Math.Pow(vals_ref[k] - vals[k], 2.0);
            norm += Math.Pow(vals_ref[k], 2.0);
        }
        err = Math.Sqrt(err);
        norm = Math.Sqrt(norm);

        if (!errCL_abs.ContainsKey(name)){
            errCL_abs[name] = new List<double>();
            normCL_ref[name] = new List<double>();
            errCL_rel[name] = new List<double>();
        }
        errCL_abs[name].Add(err);
        normCL_ref[name].Add(norm);
        errCL_rel[name].Add(err/norm);
    }
}

for(int i = 0; i < dataEvap_Ref.Count(); i++){
    for(int j = 0; j < dataEvap_Ref[i].dataGroups.Count() & j < 4; j++){
        string name = dataEvap_Ref[i].dataGroups[j].Name;
        var vals_ref = dataEvap_Ref[i].dataGroups[j].Values;
        var vals = dataEvap[i].dataGroups[j].Values;
        double norm = 0.0, err = 0.0;
        for(int k = 0; k< N_end; k++){
            err += Math.Pow(vals_ref[k] - vals[k], 2.0);
            norm += Math.Pow(vals_ref[k], 2.0);
        }
        err = Math.Sqrt(err);
        norm = Math.Sqrt(norm);
        
        if (!errEvap_abs.ContainsKey(name)){
            errEvap_abs[name] = new List<double>();
            normEvap_ref[name] = new List<double>();
            errEvap_rel[name] = new List<double>();
        }
        errEvap_abs[name].Add(err);
        normEvap_ref[name].Add(norm);
        errEvap_rel[name].Add(err/norm);
    }
}

Set threshhold for relative errors and assert its been reached

In [ ]:
double thrsh = 0.01;

foreach(var rel_errs in errCL_rel){
    foreach(var err in rel_errs.Value){
        if(err > thrsh){
            Console.WriteLine($"\x1b[31mAssertion failed in {rel_errs.Key}!\x1b[0m");
            Errors++;
        }
    }
}

foreach(var rel_errs in errEvap_rel){
    foreach(var err in rel_errs.Value){
        if(err > thrsh){
            Console.WriteLine($"\x1b[31mAssertion failed in {rel_errs.Key}!\x1b[0m");
            Errors++;
        }
    }
}

#### Print out the result & and save to file

This file can e.g. be used to determine the outcome of this sheets execution

In [ ]:
if(Errors != 0){
    Console.WriteLine($"\x1b[31mVerification failed with {Errors} errors!\x1b[0m");
}else{
    Console.WriteLine($"\x1b[32mVerification successfull!\x1b[0m");
}

In [ ]:
File.WriteAllText(".\\HeatedWallSimple_VerificationResult.txt", Errors.ToString());